# Ex3: Spark DataFrames - walmart_stock

There are some basic questions about some stock market data, in this case Walmart Stock from the years 2012-2017. 
#### Use the walmart_stock.csv file to Answer and complete the  tasks below!

#### Start a simple Spark Session

In [1]:
!apt update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-2.4.0/spark-2.4.0-bin-hadoop2.7.tgz
!tar -xvf spark-2.4.0-bin-hadoop2.7.tgz
!pip install -q findspark
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.0-bin-hadoop2.7"
import findspark
findspark.init()
# this code has to be in the first line

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Ign:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:5 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:10 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:11 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Hit:12 http://ppa.launchpad.net/cran/libgit2/u

In [2]:
from google.colab import drive
drive.mount("/content/gdrive", force_remount=True)
# this code has to be in the second line

Mounted at /content/gdrive


In [3]:
%cd '/content/gdrive/My Drive/LDS9_K273_ONLINE_BuiThiHuong/LDS9_K273_BuiThiHuong_Bai_tap_cong_diem/'

/content/gdrive/My Drive/LDS9_K273_ONLINE_BuiThiHuong/LDS9_K273_BuiThiHuong_Bai_tap_cong_diem


In [4]:
# import findspark
# findspark.init()

In [5]:
import pyspark
from pyspark import SparkContext
from pyspark.sql import SparkSession
from time import time
import numpy as np

from pyspark.sql.types import *
import pyspark.sql.functions as F
from pyspark.sql.functions import year, month

In [6]:
sc =SparkContext(master="local", appName="New Spark Context")
# just run this code 1 time
# master: IP of master computer
# just edit "master" and "appName" parameters

In [7]:
spark = SparkSession(sc)

#### Load the Walmart Stock CSV File, have Spark infer the data types.

In [8]:
stocks = spark.read.csv('walmart_stock.csv', header=True, inferSchema=True)

In [9]:
stocks.show(3)

+-------------------+------------------+---------+---------+------------------+--------+------------------+
|               Date|              Open|     High|      Low|             Close|  Volume|         Adj Close|
+-------------------+------------------+---------+---------+------------------+--------+------------------+
|2012-01-03 00:00:00|         59.970001|61.060001|59.869999|         60.330002|12668800|52.619234999999996|
|2012-01-04 00:00:00|60.209998999999996|60.349998|59.470001|59.709998999999996| 9593300|         52.078475|
|2012-01-05 00:00:00|         59.349998|59.619999|58.369999|         59.419998|12768200|         51.825539|
+-------------------+------------------+---------+---------+------------------+--------+------------------+
only showing top 3 rows



In [10]:
stocks.printSchema()

root
 |-- Date: timestamp (nullable = true)
 |-- Open: double (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Close: double (nullable = true)
 |-- Volume: integer (nullable = true)
 |-- Adj Close: double (nullable = true)



#### What are the column names?

In [11]:
stocks.columns

['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Adj Close']

#### What does the Schema look like?

In [12]:
stocks.printSchema()

root
 |-- Date: timestamp (nullable = true)
 |-- Open: double (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Close: double (nullable = true)
 |-- Volume: integer (nullable = true)
 |-- Adj Close: double (nullable = true)



#### Print out the first 5 rows.

In [13]:
stocks.show(5)

+-------------------+------------------+---------+---------+------------------+--------+------------------+
|               Date|              Open|     High|      Low|             Close|  Volume|         Adj Close|
+-------------------+------------------+---------+---------+------------------+--------+------------------+
|2012-01-03 00:00:00|         59.970001|61.060001|59.869999|         60.330002|12668800|52.619234999999996|
|2012-01-04 00:00:00|60.209998999999996|60.349998|59.470001|59.709998999999996| 9593300|         52.078475|
|2012-01-05 00:00:00|         59.349998|59.619999|58.369999|         59.419998|12768200|         51.825539|
|2012-01-06 00:00:00|         59.419998|59.450001|58.869999|              59.0| 8069400|          51.45922|
|2012-01-09 00:00:00|         59.029999|59.549999|58.919998|             59.18| 6679300|51.616215000000004|
+-------------------+------------------+---------+---------+------------------+--------+------------------+
only showing top 5 rows



#### Use describe() to learn about the DataFrame.

In [14]:
stocks.describe().show()

+-------+------------------+-----------------+-----------------+-----------------+-----------------+-----------------+
|summary|              Open|             High|              Low|            Close|           Volume|        Adj Close|
+-------+------------------+-----------------+-----------------+-----------------+-----------------+-----------------+
|  count|              1258|             1258|             1258|             1258|             1258|             1258|
|   mean| 72.35785375357709|72.83938807631165| 71.9186009594594|72.38844998012726|8222093.481717011|67.23883848728146|
| stddev|  6.76809024470826|6.768186808159218|6.744075756255496|6.756859163732991|  4519780.8431556|6.722609449996857|
|    min|56.389998999999996|        57.060001|        56.299999|        56.419998|          2094900|        50.363689|
|    max|         90.800003|        90.970001|            89.25|        90.470001|         80898100|84.91421600000001|
+-------+------------------+-----------------+--

#### There are too many decimal places for mean and stddev in the describe() dataframe. Format the numbers to just show up to two decimal places. Pay careful attention to the datatypes that .describe() returns, we didn't cover how to do this exact formatting, but we covered something very similar. [Check this link for a hint](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.Column.cast)

In [15]:
stocks.columns

['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Adj Close']

In [16]:
lst1 = ['Open', 'High', 'Low', 'Close', 'Volume']

In [17]:
stocks.printSchema()

root
 |-- Date: timestamp (nullable = true)
 |-- Open: double (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Close: double (nullable = true)
 |-- Volume: integer (nullable = true)
 |-- Adj Close: double (nullable = true)



In [18]:
# calculate statistics metrics
lst_count=[]
lst_mean=[]
lst_stddev=[]
lst_min=[]
lst_max=[]
for i in lst1:
  # lst_count.append(stocks.select(i).count())
  # lst_mean.append(stocks.select(i).groupBy().avg().collect()[0]['avg('+i+')'])
  lst_count.append(float(stocks.agg({i:'count'}).collect()[0]['count('+i+')']))
  lst_mean.append(float(stocks.agg({i:'avg'}).collect()[0]['avg('+i+')']))
  lst_stddev.append(float(stocks.agg({i:'stddev'}).collect()[0]['stddev('+i+')']))
  lst_min.append(float(stocks.agg({i:'min'}).collect()[0]['min('+i+')']))
  lst_max.append(float(stocks.agg({i:'max'}).collect()[0]['max('+i+')']))

In [19]:
# import pandas as pd
headers = ['summary','count','mean','stddev','min','max']
summary = [lst1, lst_count, lst_mean, lst_stddev, lst_min, lst_max]

# pandas_df = pd.DataFrame.from_dict(dict(zip(headers, summary)))
# stocks_describle = spark.createDataFrame(pandas_df)
# stocks_describle.show()

In [20]:
summary

[['Open', 'High', 'Low', 'Close', 'Volume'],
 [1258.0, 1258.0, 1258.0, 1258.0, 1258.0],
 [72.35785375357709,
  72.83938807631165,
  71.9186009594594,
  72.38844998012726,
  8222093.481717011],
 [6.76809024470826,
  6.768186808159218,
  6.744075756255496,
  6.756859163732991,
  4519780.8431556],
 [56.389998999999996, 57.060001, 56.299999, 56.419998, 2094900.0],
 [90.800003, 90.970001, 89.25, 90.470001, 80898100.0]]

In [21]:
# create data with list of tuples
lst_tpl=[]
for i in range(len(headers[1:])):
  for j in range(len(lst1)):
    tpl = (headers[i+1], summary[i+1][0], summary[i+1][1], summary[i+1][2], summary[i+1][3], summary[i+1][4])
  lst_tpl.append(tpl) 
print(lst_tpl)

[('count', 1258.0, 1258.0, 1258.0, 1258.0, 1258.0), ('mean', 72.35785375357709, 72.83938807631165, 71.9186009594594, 72.38844998012726, 8222093.481717011), ('stddev', 6.76809024470826, 6.768186808159218, 6.744075756255496, 6.756859163732991, 4519780.8431556), ('min', 56.389998999999996, 57.060001, 56.299999, 56.419998, 2094900.0), ('max', 90.800003, 90.970001, 89.25, 90.470001, 80898100.0)]


In [22]:
# create dataframe from list of tuples
stocks_describle  = spark.createDataFrame(lst_tpl, ['summary']+lst1)
stocks_describle.show()

+-------+------------------+-----------------+-----------------+-----------------+-----------------+
|summary|              Open|             High|              Low|            Close|           Volume|
+-------+------------------+-----------------+-----------------+-----------------+-----------------+
|  count|            1258.0|           1258.0|           1258.0|           1258.0|           1258.0|
|   mean| 72.35785375357709|72.83938807631165| 71.9186009594594|72.38844998012726|8222093.481717011|
| stddev|  6.76809024470826|6.768186808159218|6.744075756255496|6.756859163732991|  4519780.8431556|
|    min|56.389998999999996|        57.060001|        56.299999|        56.419998|        2094900.0|
|    max|         90.800003|        90.970001|            89.25|        90.470001|        8.08981E7|
+-------+------------------+-----------------+-----------------+-----------------+-----------------+



In [23]:
# change data type and format
lst_2places = ['Open','High','Low','Close']
lst_1place = ['Volume']
for col_name in lst_2places:
    stocks_describle = stocks_describle.withColumn(col_name, F.format_number(col_name,2))
for col_name in lst_1place:
    stocks_describle = stocks_describle.withColumn(col_name, stocks_describle[col_name].cast(IntegerType()))

In [24]:
# final result
stocks_describle.show()

+-------+--------+--------+--------+--------+--------+
|summary|    Open|    High|     Low|   Close|  Volume|
+-------+--------+--------+--------+--------+--------+
|  count|1,258.00|1,258.00|1,258.00|1,258.00|    1258|
|   mean|   72.36|   72.84|   71.92|   72.39| 8222093|
| stddev|    6.77|    6.77|    6.74|    6.76| 4519780|
|    min|   56.39|   57.06|   56.30|   56.42| 2094900|
|    max|   90.80|   90.97|   89.25|   90.47|80898100|
+-------+--------+--------+--------+--------+--------+



#### Create a new dataframe with a column called HV Ratio that is the ratio of the High Price versus volume of stock traded for a day.

In [25]:
HV_ratio = stocks.withColumn('HV ratio',stocks.High/stocks.Volume).select('HV ratio')
HV_ratio.show()

+--------------------+
|            HV ratio|
+--------------------+
|4.819714653321546E-6|
|6.290848613094555E-6|
|4.669412994783916E-6|
|7.367338463826307E-6|
|8.915604778943901E-6|
|8.644477436914568E-6|
|9.351828421515645E-6|
| 8.29141562102703E-6|
|7.712212102001476E-6|
|7.071764823529412E-6|
|1.015495466386981E-5|
|6.576354146362592...|
| 5.90145296180676E-6|
|8.547679455011844E-6|
|8.420709512685392E-6|
|1.041448341728929...|
|8.316075414862431E-6|
|9.721183814992126E-6|
|8.029436027707578E-6|
|6.307432259386365E-6|
+--------------------+
only showing top 20 rows



#### What day had the Peak High in Price?

In [26]:
stocks.where(stocks.High == stocks.groupBy().max('High').collect()[0]['max(High)']).select('Date').show()

+-------------------+
|               Date|
+-------------------+
|2015-01-13 00:00:00|
+-------------------+



#### What is the mean of the Close column?

In [27]:
stocks.select('Close').groupBy().avg().show()

+-----------------+
|       avg(Close)|
+-----------------+
|72.38844998012726|
+-----------------+



#### What is the max and min of the Volume column?

In [28]:
stocks.groupBy().agg(F.max('Volume'), F.min('Volume')).show()

+-----------+-----------+
|max(Volume)|min(Volume)|
+-----------+-----------+
|   80898100|    2094900|
+-----------+-----------+



#### How many days was the Close lower than 60 dollars?

In [29]:
stocks.where(stocks.Close < 60).select('Date').distinct().count()

81

#### What percentage of the time was the High greater than 80 dollars ?
#### In other words, (Number of Days High>80)/(Total Days in the dataset)

In [30]:
number_of_days_High_80 = stocks.filter(stocks.High > 80).select('Date').distinct().count()

In [31]:
total_of_days = stocks.select('Date').distinct().count()

In [32]:
number_of_days_High_80/total_of_days*100

9.141494435612083

#### What is the Pearson correlation between High and Volume?
#### [Hint](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrameStatFunctions.corr)

In [33]:
stocks.corr('High','Volume')

-0.3384326061737161

#### What is the max High per year?

In [34]:
stocks.printSchema()

root
 |-- Date: timestamp (nullable = true)
 |-- Open: double (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Close: double (nullable = true)
 |-- Volume: integer (nullable = true)
 |-- Adj Close: double (nullable = true)



In [35]:
# create new column named 'Year'
stocks = stocks.withColumn('Year',year(stocks.Date))

In [36]:
stocks.groupBy('Year').max('High').show()

+----+---------+
|Year|max(High)|
+----+---------+
|2015|90.970001|
|2013|81.370003|
|2014|88.089996|
|2012|77.599998|
|2016|75.190002|
+----+---------+



#### What is the average Close for each Calendar Month?
#### In other words, across all the years, what is the average Close price for Jan,Feb, Mar, etc... Your result will have a value for each of these months. 

In [37]:
# create new column named 'Month'
stocks = stocks.withColumn('Month',month(stocks.Date))

In [38]:
stocks.groupBy('Month').avg('Close').sort('Month').show()

+-----+-----------------+
|Month|       avg(Close)|
+-----+-----------------+
|    1|71.44801958415842|
|    2|  71.306804443299|
|    3|71.77794377570092|
|    4|72.97361900952382|
|    5|72.30971688679247|
|    6| 72.4953774245283|
|    7|74.43971943925233|
|    8|73.02981855454546|
|    9|72.18411785294116|
|   10|71.57854545454543|
|   11| 72.1110893069307|
|   12|72.84792478301885|
+-----+-----------------+

